In [84]:
import io
import json
import sqlite3 as sql
import pandas as pd
import os

def query(query_string, connection):
    return pd.read_sql_query(query_string, connection)

In [85]:
file = open('owl.sql', 'w+')

In [86]:
json_file = open('matches.json')
data = json.load(json_file)['content']
json_file.close()

In [87]:
match_schema = (
    'CREATE TABLE [MATCHES] ( '
    'MATCH_ID INT, '
    'TEAM_ONE_ID INT, '
    'TEAM_TWO_ID INT, '
    'TEAM_ONE_SCORE INT, '
    'TEAM_TWO_SCORE INT, '
    'WINNER_ID INT, '
    'TIMESTAMP DATETIME, '
    'PRIMARY KEY (MATCH_ID)'
    ');' 
)

In [88]:
file.write(match_schema + '\n\n')

174

In [89]:
for match in data:
    insert = (
        'INSERT INTO MATCHES '
        '(MATCH_ID, TEAM_ONE_ID, TEAM_TWO_ID, '
        'TEAM_ONE_SCORE, TEAM_TWO_SCORE, '
        'WINNER_ID, TIMESTAMP)'
        'VALUES (%d, %d, %d, %d, %d, %d, %d);'
        % (match['id'], match['competitors'][0]['id'],
          match['competitors'][1]['id'],
          match['scores'][0]['value'],
          match['scores'][1]['value'],
          match['winner']['id'],
          match['actualStartDate'])
    )
    file.write(insert+'\n')

In [90]:
#connection = sql.connect('test.db')

In [91]:
#cursor = connection.cursor()
#db = io.open('owl.sql', 'r', encoding='ISO-8859-1')
#database = db.read()
#cursor.executescript(database)

In [92]:
game_schema = (
    '\nCREATE TABLE [GAMES] ('
    'GAME_ID INT, '
    'MATCH_ID INT, '
    'MAP TEXT, '
    'PRIMARY KEY (GAME_ID)'
    ');\n\n'
)

file.write(game_schema)

85

In [93]:
for match in data:
    for game in match['games']:
        try:
            map_name = str(game['attributes']['map']).replace('-', ' ')
            insert = (
                'INSERT INTO GAMES (GAME_ID, MATCH_ID, MAP) VALUES '
                '(%d, %d, \'%s\');'
                % (game['id'], match['id'], map_name)
            )
            file.write(insert+'\n')
        except:
            print('Error')

Error
Error


In [94]:
#try:
#    connection = sql.connect('test.db')
#    cursor = connection.cursor()
#    db = io.open('owl.sql', 'r', encoding='ISO-8859-1')
#    database = db.read()
#    cursor.executescript(database)
#except Exception as e:
#    print(e)
#    os.remove('test.db')

In [95]:
teams_schema = (
    '\nCREATE TABLE [TEAMS] ('
    'TEAM_ID INT, '
    'TEAM_NAME TEXT, '
    'TEAM_LOCATION TEXT, '
    'PRIMARY KEY (TEAM_ID)'
    ');\n\n'
)

file.write(teams_schema)

97

In [96]:
team_list = []
for match in data:
    for competitor in match['competitors']:
        if competitor['id'] not in team_list:
            team_list.append(competitor['id'])
            insert = (
                    'INSERT INTO TEAMS (TEAM_ID, TEAM_NAME, TEAM_LOCATION) VALUES '
                    '(%d, \'%s\', \'%s\');'
                    % (competitor['id'], 
                       competitor['name'], 
                       competitor['homeLocation'])
                )
            file.write(insert+'\n')

In [97]:
#try:
#    connection = sql.connect('test.db')
#    cursor = connection.cursor()
#    db = io.open('owl.sql', 'r', encoding='ISO-8859-1')
#    database = db.read()
#    cursor.executescript(database)
#except Exception as e:
#    print(e)
#    os.remove('test.db')

In [98]:
teams_schema = (
    '\nCREATE TABLE [PLAYERS] ('
    'COMPETITOR_ID INT,'
    'TEAM_ID INT, '
    'HANDLE TEXT, '
    'ROLE TEXT, '
    'NUMBER INT,'
    'NAME TEXT,'
    'PHOTO TEXT,'
    'PRIMARY KEY (COMPETITOR_ID)'
    ');\n\n'
)

file.write(teams_schema)

143

In [99]:
data[0]['competitors'][0]['players'][0]['player']['attributes']['role']

'tank'

In [101]:
team_list = []
player_list = []
for match in data:
    for competitor in match['competitors']:
        if competitor['id'] not in team_list:
            team_list.append(competitor['id'])
            for player in competitor['players']:
                if player['player']['id'] not in player_list:
                    player_list.append(player['player']['id'])
                    insert = (
                            'INSERT INTO PLAYERS (COMPETITOR_ID, TEAM_ID, HANDLE, ROLE, NUMBER, NAME, PHOTO) VALUES '
                            '(%d, %d, \'%s\', \'%s\', %d, \'%s\', \'%s\');'
                            % (player['player']['id'], 
                               player['team']['id'], 
                               player['player']['name'],
                               player['player']['attributes']['role'],
                               player['player']['attributes']['player_number'],
                               ' '.join([player['player']['givenName'], player['player']['familyName']]),
                               player['player']['headshot'])
                        )
                    file.write(insert+'\n')

In [102]:
file.close()

In [103]:
try:
    connection = sql.connect('test.db')
    cursor = connection.cursor()
    db = io.open('owl.sql', 'r', encoding='ISO-8859-1')
    database = db.read()
    cursor.executescript(database)
except Exception as e:
    print(e)
    os.remove('test.db')

In [104]:
query('select handle, team_name, number, role from players natural join teams where team_name=\'Dallas Fuel\' order by role', connection)

,HANDLE,TEAM_NAME,NUMBER,ROLE
0,aKm,Dallas Fuel,10,offense
1,Taimou,Dallas Fuel,13,offense
2,ZachaREEE,Dallas Fuel,22,offense
3,HarryHook,Dallas Fuel,9,support
4,uNKOE,Dallas Fuel,24,support
5,Closer,Dallas Fuel,17,support
6,OGE,Dallas Fuel,2,tank
7,Mickie,Dallas Fuel,7,tank
8,NotE,Dallas Fuel,3,tank
9,Trill,Dallas Fuel,30,tank
